This file grabs everything from the LSMS survey that I think an image could possibly recognize and uses those features to predict consumption. This serves as a "gold standard" for any image-based model. It turns out that the CNN model performs almost as well as this gold standard on all the output metrics!

In [1]:
import pandas as pd
import numpy as np
import os

import sys
sys.path.append('..')
from utils import merge_on_lat_lon

In [2]:
df_clusters = pd.read_csv('../LSMS/output/malawi/cluster_data.csv')

In [3]:
df_cons = pd.read_stata('../LSMS/input/malawi/IHS4 Consumption Aggregate.dta')[['case_id', 'ea_id']]

In [4]:
df_geo = pd.read_stata('../LSMS/input/malawi/HouseholdGeovariables_stata11/HouseholdGeovariablesIHS4.dta')
df_hhf = pd.read_stata('../LSMS/input/malawi/HH_MOD_F.dta')
df_plot = pd.read_stata('../LSMS/input/malawi/PlotGeovariablesIHS4.dta')

In [5]:
df_com = pd.read_stata('../LSMS/input/malawi/COM_CD.dta')
df_com2 = pd.read_stata('../LSMS/input/malawi/COM_CF1.dta')

In [6]:
# rooms = df_hhf['hh_f10']
# roof = df_hhf['hh_f08']

# # all distance infrasturcture metrics
# road_type = df_com['com_cd01']
# dist_daily_market = df_com['com_cd16']
# dist_larger_weekly = df_com['com_cd18a']
# dist_perm_admarc = df_com['com_cd20a']
# dist_post_office = df_com['com_cd22a']
# dist_telephone = df_com['com_cd24a']
# dist_gov_prim_school = df_com['com_cd27a']
# dist_gov_sec_school = df_com['com_cd36a']
# dist_comm_sec_school = df_com['com_cd40a']
# dist_medicines = df_com['com_cd49a']
# dist_health_clinic = df_com['com_cd51a']
# dist_doctor = df_com['com_cd60a']
# dist_bank = df_com['com_cd67a']
# dist_microfinance = df_com['com_cd69a']

# dist_agric_exten_officer = df_com2['com_cf08a']

# dist_admarc_outlet = df_geo['dist_admarc']
# dist_agric_market = df_geo['dist_agmrkt']
# dist_tobacco_auction = df_geo['dist_auction']
# dist_boma = df_geo['dist_boma']
# dist_border = df_geo['dist_borderpost']
# dist_popcenter = df_geo['dist_popcenter']
# dist_road = df_geo['dist_road']

# dist_hh = df_plot['dist_hh']

# # temp
# mean_temp = df_geo['af_bio_1']
# mean_temp_wet_q = df_geo['af_bio_8']

# # rain
# mean_rain = df_geo['af_bio_12']
# mean_rain_wet_month = df_geo['af_bio_13']
# mean_rain_wet_q = df_geo['af_bio_16']

In [7]:
hhf_input = df_hhf[['case_id', 'HHID', 'hh_f10', 'hh_f08']]
com_input = df_com[['ea_id', 'com_cd01', 'com_cd16', 'com_cd18a', 'com_cd20a', 'com_cd22a', 'com_cd24a',
                   'com_cd27a', 'com_cd36a', 'com_cd40a', 'com_cd49a', 'com_cd51a', 'com_cd60a', 'com_cd67a',
                   'com_cd69a']]

com2_input = df_com2[['ea_id', 'com_cf08a']]

geo_input = df_geo[['case_id', 'HHID', 'dist_admarc', 'dist_agmrkt', 'dist_auction', 'dist_boma', 'dist_borderpost',
                  'dist_popcenter', 'dist_road', 'af_bio_1', 'af_bio_8', 'af_bio_12', 'af_bio_13', 'af_bio_16']]

plot_input = df_plot[['case_id', 'HHID', 'dist_hh']]

In [8]:
df_merge = pd.merge(df_cons, hhf_input, on='case_id', how='left')
print(df_merge.shape)
df_merge = pd.merge(df_merge, com_input, on='ea_id', how='left')
print(df_merge.shape)
df_merge = pd.merge(df_merge, com2_input, on='ea_id', how='left')
print(df_merge.shape)
df_merge = pd.merge(df_merge, geo_input, on=['case_id', 'HHID'], how='left')
print(df_merge.shape)
df_merge = pd.merge(df_merge, plot_input, on=['case_id', 'HHID'], how='left')
print(df_merge.shape)

(12447, 5)
(12447, 19)
(12447, 20)
(12447, 32)
(19870, 33)


In [9]:
df_merge.head()

case_id     ea_id                              HHID  hh_f10  \
0  301025230225  30102523  0001c970eecf473099368557e2080b3e       2   
1  210374850204  21037485  000509f5cfcc4b078a09672b09425e95       3   
2  311057710075  31105771  000bc107780044e59327dbf7ec960ac1       3   
3  311057710075  31105771  000bc107780044e59327dbf7ec960ac1       3   
4  311057710075  31105771  000bc107780044e59327dbf7ec960ac1       3   

        hh_f08         com_cd01  com_cd16  com_cd18a  com_cd20a  com_cd22a  \
0        GRASS  GRADED GRAVELED      10.0       10.0       10.0       10.0   
1  IRON SHEETS      TAR/ASPHALT       NaN        NaN        NaN        NaN   
2        GRASS              NaN       NaN        NaN        NaN        NaN   
3        GRASS              NaN       NaN        NaN        NaN        NaN   
4        GRASS              NaN       NaN        NaN        NaN        NaN   

   ...  dist_boma  dist_borderpost  dist_popcenter  dist_road  af_bio_1  \
0  ...       44.0             39.0            32.0       13.0     220.0   
1  ...        3.0             74.0             3.0        2.0     200.0   
2  ...       12.0             36.0            12.0        1.0     257.0   
3  ...       12.0             36.0            12.0        1.0     257.0   
4  ...       12.0             36.0            12.0        1.0     257.0   

   af_bio_8  af_bio_12  af_bio_13  af_bio_16  dist_hh  
0     234.0      944.0      238.0      660.0      0.2  
1     218.0      863.0      224.0      599.0      NaN  
2     280.0      937.0      199.0      549.0      1.1  
3     280.0      937.0      199.0      549.0      1.1  
4     280.0      937.0      199.0      549.0      1.2  

[5 rows x 33 columns]

In [10]:
df_cords = df_geo[['case_id', 'HHID', 'lat_modified', 'lon_modified']]
df_merge = pd.merge(df_merge, df_cords, on=['case_id', 'HHID'], how='left')

In [11]:
df_merge.shape

(19870, 35)

In [12]:
df_merge.rename(columns={'lat_modified': 'cluster_lat', 'lon_modified': 'cluster_lon'}, inplace=True)

In [13]:
df_merge.dropna(subset=['cluster_lat', 'cluster_lon'], inplace=True)
df_merge.shape

(19865, 35)

In [14]:
df_final = merge_on_lat_lon(df_clusters, df_merge)

In [16]:
df_final.head()

cluster_lat  cluster_lon  cluster_persons_surveyed  \
0    -17.09515    35.217213                        79   
1    -17.09515    35.217213                        79   
2    -17.09515    35.217213                        79   
3    -17.09515    35.217213                        79   
4    -17.09515    35.217213                        79   

   cluster_annual_consumption_pc  cluster_annual_phone_consumption_pc  \
0                     961.328026                            47.627469   
1                     961.328026                            47.627469   
2                     961.328026                            47.627469   
3                     961.328026                            47.627469   
4                     961.328026                            47.627469   

   cluster_cellphones_pc  cluster_estimated_annual_phone_cost_pc  \
0               0.177215                              428.481013   
1               0.177215                              428.481013   
2               0.177215                              428.481013   
3               0.177215                              428.481013   
4               0.177215                              428.481013   

   cluster_nightlights       case_id     ea_id  ... dist_boma  \
0                  0.0  311017590042  31101759  ...      21.0   
1                  0.0  311017590010  31101759  ...      20.0   
2                  0.0  311017590064  31101759  ...      20.0   
3                  0.0  311017590064  31101759  ...      20.0   
4                  0.0  311017590146  31101759  ...      20.0   

   dist_borderpost dist_popcenter dist_road  af_bio_1  af_bio_8  af_bio_12  \
0              4.0           21.0       1.0     258.0     281.0      837.0   
1              4.0           20.0       0.0     258.0     281.0      837.0   
2              4.0           20.0       0.0     258.0     281.0      834.0   
3              4.0           20.0       0.0     258.0     281.0      834.0   
4              5.0           20.0       1.0     257.0     281.0      840.0   

   af_bio_13  af_bio_16  dist_hh  
0      182.0      494.0      1.2  
1      182.0      494.0      1.0  
2      181.0      492.0      1.7  
3      181.0      492.0      1.7  
4      182.0      495.0      0.9  

[5 rows x 41 columns]

In [17]:
df_final.shape

(19865, 41)

In [18]:
df_use = df_final.drop(['case_id', 'ea_id', 'HHID'], axis=1)

In [19]:
df_use = pd.get_dummies(df_use)

In [20]:
clusters = df_use.groupby(['cluster_lat', 'cluster_lon'])

In [21]:
cluster_df = clusters.mean().reset_index()

In [22]:
cluster_df

cluster_lat  cluster_lon  cluster_persons_surveyed  \
0     -17.095150    35.217213                        79   
1     -17.092351    35.114643                        70   
2     -17.016698    35.079629                        78   
3     -16.977243    35.205706                        66   
4     -16.956385    35.168967                        61   
..           ...          ...                       ...   
775    -9.591378    33.057450                        62   
776    -9.550397    33.291558                        59   
777    -9.519230    33.139193                        72   
778    -9.507538    33.259649                        63   
779    -9.429667    33.022118                        56   

     cluster_annual_consumption_pc  cluster_annual_phone_consumption_pc  \
0                       961.328026                            47.627469   
1                       855.258482                             3.189638   
2                      1058.343450                             1.978659   
3                      1127.493134                             8.631155   
4                       736.167585                             5.081308   
..                             ...                                  ...   
775                     952.339970                             1.452034   
776                     839.451073                            24.716671   
777                    1218.595595                            35.439080   
778                    1210.222098                            27.476154   
779                    1036.615583                            21.935617   

     cluster_cellphones_pc  cluster_estimated_annual_phone_cost_pc  \
0                 0.177215                              428.481013   
1                 0.028571                               32.571429   
2                 0.025641                               19.230769   
3                 0.045455                               83.333333   
4                 0.065574                               49.180328   
..                     ...                                     ...   
775               0.016129                               16.129032   
776               0.050847                              277.966102   
777               0.166667                              350.000000   
778               0.158730                              266.666667   
779               0.160714                              242.857143   

     cluster_nightlights    hh_f10  com_cd16  ...  hh_f08_GRASS  \
0               0.000000  2.541667       NaN  ...      0.666667   
1               0.000000  2.157895       5.0  ...      0.789474   
2               0.000000  2.333333      60.0  ...      0.571429   
3               0.121212  2.264706       2.0  ...      0.705882   
4               0.502674  2.045455      18.0  ...      0.454545   
..                   ...       ...       ...  ...           ...   
775             0.000000  3.500000       2.0  ...      0.444444   
776             0.000000  2.851852       NaN  ...      0.407407   
777             0.000000  2.971429       NaN  ...      0.371429   
778             0.000000  3.000000       NaN  ...      0.255814   
779             0.000000  2.947368       5.0  ...      0.526316   

     hh_f08_IRON SHEETS  hh_f08_CLAY TILES  hh_f08_CONCRETE  \
0              0.333333                0.0              0.0   
1              0.210526                0.0              0.0   
2              0.428571                0.0              0.0   
3              0.294118                0.0              0.0   
4              0.545455                0.0              0.0   
..                  ...                ...              ...   
775            0.555556                0.0              0.0   
776            0.592593                0.0              0.0   
777            0.628571                0.0              0.0   
778            0.744186                0.0              0.0   
779            0.473684                0.0              

In [23]:
# a few columns have a high percentage of NA
cluster_df.isna().sum() / len(cluster_df)

cluster_lat                               0.000000
cluster_lon                               0.000000
cluster_persons_surveyed                  0.000000
cluster_annual_consumption_pc             0.000000
cluster_annual_phone_consumption_pc       0.000000
cluster_cellphones_pc                     0.000000
cluster_estimated_annual_phone_cost_pc    0.000000
cluster_nightlights                       0.000000
hh_f10                                    0.000000
com_cd16                                  0.379487
com_cd18a                                 0.350000
com_cd20a                                 0.257692
com_cd22a                                 0.151282
com_cd24a                                 0.191026
com_cd27a                                 0.032051
com_cd36a                                 0.032051
com_cd40a                                 0.032051
com_cd49a                                 0.603846
com_cd51a                                 0.288462
com_cd60a                      

# Modeling

In [24]:
# This is a bunch of code from the Jean et al Github that is modified to work with Python3 and our data

import numpy as np
import pandas as pd
import random
from scipy import stats
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
import sklearn.linear_model as linear_model
import matplotlib.pyplot as plt
from matplotlib.collections import EllipseCollection
import seaborn as sns


def predict_consumption(
    X, y, k=5, k_inner=5, points=10,
        alpha_low=1, alpha_high=5, margin=0.25):
    """
    Plots predicted consumption
    """
    y_hat, r2 = run_cv(X, y, k, k_inner, points, alpha_low, alpha_high)
    return X, y, y_hat, r2


def run_cv(X, y, k, k_inner, points, alpha_low, alpha_high, randomize=False):
    """
    Runs nested cross-validation to make predictions and compute r-squared.
    """
    alphas = np.logspace(alpha_low, alpha_high, points)
    r2s = np.zeros((k,))
    y_hat = np.zeros_like(y)
    kf = KFold(n_splits=k, shuffle=True)
    fold = 0
    for train_idx, test_idx in kf.split(X):
        r2s, y_hat, fold = evaluate_fold(
            X, y, train_idx, test_idx, k_inner, alphas, r2s, y_hat, fold,
            randomize)
    return y_hat, r2s.mean()


def scale_features(X_train, X_test):
    """
    Scales features using StandardScaler.
    """
    X_scaler = StandardScaler(with_mean=True, with_std=False)
    X_train = X_scaler.fit_transform(X_train)
    X_test = X_scaler.transform(X_test)
    return X_train, X_test


def train_and_predict_ridge(alpha, X_train, y_train, X_test):
    """
    Trains ridge model and predicts test set.
    """
    ridge = linear_model.Ridge(alpha)
    ridge.fit(X_train, y_train)
    y_hat = ridge.predict(X_test)
    return y_hat


def predict_inner_test_fold(X, y, y_hat, train_idx, test_idx, alpha):
    """
    Predicts inner test fold.
    """
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    X_train, X_test = scale_features(X_train, X_test)
    y_hat[test_idx] = train_and_predict_ridge(alpha, X_train, y_train, X_test)
    return y_hat


def find_best_alpha(X, y, k_inner, alphas):
    """
    Finds the best alpha in an inner CV loop.
    """
    kf = KFold(n_splits=k_inner, shuffle=True)
    best_alpha = 0
    best_r2 = 0
    for idx, alpha in enumerate(alphas):
        y_hat = np.zeros_like(y)
        for train_idx, test_idx in kf.split(X):
            y_hat = predict_inner_test_fold(
                X, y, y_hat, train_idx, test_idx, alpha)
        r2 = stats.pearsonr(y, y_hat)[0] ** 2
        if r2 > best_r2:
            best_alpha = alpha
            best_r2 = r2
    print('best alpha', best_alpha)
    return best_alpha


def evaluate_fold(
    X, y, train_idx, test_idx, k_inner, alphas, r2s, y_hat, fold,
        randomize):
    """
    Evaluates one fold of outer CV.
    """
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    if randomize:
        random.shuffle(y_train)
    best_alpha = find_best_alpha(X_train, y_train, k_inner, alphas)
    X_train, X_test = scale_features(X_train, X_test)
    y_test_hat = train_and_predict_ridge(best_alpha, X_train, y_train, X_test)
    r2 = stats.pearsonr(y_test, y_test_hat)[0] ** 2
    r2s[fold] = r2
    y_hat[test_idx] = y_test_hat
    return r2s, y_hat, fold + 1


In [25]:
def nan_handler(df, ignore=list()):
    nas = df.isna().sum()
    for c in nas.index:
        if c in ignore:
            continue
        if nas[c] > 0:
            df[c] = df[c].fillna(df[c].median())
    return df

In [26]:
cleaned_df = nan_handler(cluster_df, ignore=['cons', 'cluster_annual_consumption_pc', 'cluster_annual_phone_consumption_pc', 'cluster_cellphones_pc', 'cluster_estimated_annual_phone_cost_pc'])



In [27]:
cleaned_df.head()

cluster_lat  cluster_lon  cluster_persons_surveyed  \
0   -17.095150    35.217213                        79   
1   -17.092351    35.114643                        70   
2   -17.016698    35.079629                        78   
3   -16.977243    35.205706                        66   
4   -16.956385    35.168967                        61   

   cluster_annual_consumption_pc  cluster_annual_phone_consumption_pc  \
0                     961.328026                            47.627469   
1                     855.258482                             3.189638   
2                    1058.343450                             1.978659   
3                    1127.493134                             8.631155   
4                     736.167585                             5.081308   

   cluster_cellphones_pc  cluster_estimated_annual_phone_cost_pc  \
0               0.177215                              428.481013   
1               0.028571                               32.571429   
2               0.025641                               19.230769   
3               0.045455                               83.333333   
4               0.065574                               49.180328   

   cluster_nightlights    hh_f10  com_cd16  ...  hh_f08_GRASS  \
0             0.000000  2.541667       7.0  ...      0.666667   
1             0.000000  2.157895       5.0  ...      0.789474   
2             0.000000  2.333333      60.0  ...      0.571429   
3             0.121212  2.264706       2.0  ...      0.705882   
4             0.502674  2.045455      18.0  ...      0.454545   

   hh_f08_IRON SHEETS  hh_f08_CLAY TILES  hh_f08_CONCRETE  \
0            0.333333                0.0              0.0   
1            0.210526                0.0              0.0   
2            0.428571                0.0              0.0   
3            0.294118                0.0              0.0   
4            0.545455                0.0              0.0   

   hh_f08_PLASTIC SHEETING  hh_f08_OTHER (SPECIFY)  com_cd01_TAR/ASPHALT  \
0                      0.0                     0.0                   0.0   
1                      0.0                     0.0                   0.0   
2                      0.0                     0.0                   0.0   
3                      0.0                     0.0                   0.0   
4                      0.0                     0.0                   0.0   

   com_cd01_GRADED GRAVELED  com_cd01_DIRT ROAD (MAINTAINED)  \
0                       0.0                              0.0   
1                       0.0                              1.0   
2                       0.0                              1.0   
3                       0.0                              1.0   
4                       0.0                              1.0   

   com_cd01_DIRT TRACK  
0                  0.0  
1                  0.0  
2                  0.0  
3                  0.0  
4                  0.0  

[5 rows x 46 columns]

In [28]:
y = cleaned_df['cluster_annual_consumption_pc'].values
y_log = np.log(y)

to_drop = ['cluster_lat', 'cluster_lon', 'cluster_persons_surveyed', 'cluster_annual_consumption_pc', 'cluster_annual_phone_consumption_pc', 'cluster_cellphones_pc', 'cluster_estimated_annual_phone_cost_pc', 'cluster_nightlights']
x = cleaned_df.drop(to_drop, axis=1).values


In [29]:
_, _, _, r2 = predict_consumption(x, y_log)
r2

best alpha 10.0
best alpha 10.0
best alpha 10.0
best alpha 10.0
best alpha 27.825594022071243


0.4952549885321925

In [30]:
_, _, _, r2 = predict_consumption(x, y)
r2

best alpha 27.825594022071243
best alpha 27.825594022071243
best alpha 10.0
best alpha 10.0
best alpha 10.0


0.34342279997242225

In [31]:
y = cleaned_df['cluster_annual_phone_consumption_pc'].values + 0.0001
y_log = np.log(y)

In [32]:
_, _, _, r2 = predict_consumption(x, y_log)
r2

best alpha 10.0
best alpha 10.0
best alpha 27.825594022071243
best alpha 27.825594022071243
best alpha 10.0


0.34917154357423125

In [33]:
_, _, _, r2 = predict_consumption(x, y)
r2

best alpha 10.0
best alpha 27.825594022071243
best alpha 27.825594022071243
best alpha 10.0
best alpha 10.0


0.4410696383948601

In [34]:
y = cleaned_df['cluster_cellphones_pc'].values + 0.0001
y_log = np.log(y)

In [35]:
_, _, _, r2 = predict_consumption(x, y_log)
r2

best alpha 10.0
best alpha 10.0
best alpha 10.0
best alpha 10.0
best alpha 10.0


0.4182310663640463

In [36]:
_, _, _, r2 = predict_consumption(x, y)
r2

best alpha 27.825594022071243
best alpha 10.0
best alpha 10.0
best alpha 27.825594022071243
best alpha 10.0


0.5719189853065993

In [37]:
not_nas = ~cleaned_df['cluster_estimated_annual_phone_cost_pc'].isna()

In [38]:
y = cleaned_df['cluster_estimated_annual_phone_cost_pc'].values[not_nas] + 0.0001
y_log = np.log(y)

In [39]:
_, _, y_hat_log, r2 = predict_consumption(x, y_log)
r2

best alpha 10.0
best alpha 27.825594022071243
best alpha 27.825594022071243
best alpha 77.4263682681127
best alpha 27.825594022071243


0.3312690652260189

In [40]:
_, _, y_hat, r2 = predict_consumption(x, y)
r2

best alpha 10.0
best alpha 10.0
best alpha 10.0
best alpha 27.825594022071243
best alpha 10.0


0.3602449007832077